# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-NOTEBOOK_CODE_HERE-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Nobel laureates explorative search") 


Consider the following exploratory scenario:

>  Investigate the authors who won the Nobel award for literature and get information about the nationality of the winners and analyse their literary production



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wd:Q37922`   | Nobel Prize in Literature     | node |
| `wd:Q36180`   | writer        | node |
| `wd:Q38`      | Italy         | node |
| `wd:Q3624078` | Sovereign state  | node |
| `wd:Q213678`  | Vatican Library  | node |


Also consider

```
?p wdt:P27 wd:Q38  . 
?p wdt:P106/wdt:P279* wd:Q36180  . 
```

is the BGP to retrieve all **italian authors**

The workload should


1. Identify the BGP for obtaining the nobel laureates and the nominees with their relevant attributes

2. Get the number of Italian, French and Iranian winners and check which nation won more awards in the last twenty years.

3. Determine how many Literature nobel laureates have the following contries: Italy, Germany, France, Romania, Denmark, Iran, and China

4. How many Literature Nobel award winners have a PhD (aka Doctorate of Philosophy) (you may check if they have a doctoral advisor)?

5. Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?

In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT( ?p)
WHERE { 
#?p wdt:P27 wd:Q38  . 
?p wdt:P279 wd:Q482980   . 

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '34')]


1

# Workload 1

In [123]:
# 1. Identify the BGP for obtaining the nobel laureates and the nominees with their relevant attributes

In [124]:
# To find all the nobel laureates and the nominees we find the node for the Nobel Prize (to analyze it) starting 
# from the node "Noble Prize in Literature"

In [125]:
queryString = """
SELECT ?p ?pname
WHERE { 
 
 wd:Q37922  wdt:P279 ?p   . 

# this returns the labels
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q7191'), ('pname', 'Nobel Prize')]


1

In [126]:
# Get wd:Q7191 --> 'Nobel Prize'

In [127]:
# Find list of Nobel Prize existing

In [128]:
queryString = """
SELECT DISTINCT ?s ?sname
WHERE { 
 
 ?s wdt:P279  wd:Q7191 . 

# this returns the labels
   ?s <http://schema.org/name> ?sname .
   
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q35637'), ('sname', 'Nobel Peace Prize')]
[('s', 'http://www.wikidata.org/entity/Q37922'), ('sname', 'Nobel Prize in Literature')]
[('s', 'http://www.wikidata.org/entity/Q38104'), ('sname', 'Nobel Prize in Physics')]
[('s', 'http://www.wikidata.org/entity/Q44585'), ('sname', 'Nobel Prize in Chemistry')]
[('s', 'http://www.wikidata.org/entity/Q47170'), ('sname', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
[('s', 'http://www.wikidata.org/entity/Q80061'), ('sname', 'Nobel Prize in Physiology or Medicine')]


6

In [129]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
 
 ?s ?p  wd:Q7191 . 

# this returns the labels
   ?p <http://schema.org/name> ?pname .
   
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pname', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2959'), ('pname', 'permanent duplicated item')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pname', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('pname', 'main subject')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]
[('p', 'htt

14

In [130]:
# Return on 'Noble Prize in Literature' we analiyze it to find interesting property

In [131]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
 
 ?s ?p  wd:Q37922 . 

# this returns the labels
   ?p <http://schema.org/name> ?pname .
   
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pname', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pname', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P793'), ('pname', 'significant event')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('pname', 'main subject')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]


10

In [132]:
# Get wdt:P1411 --> predicate 'nominated for' //// wdt:P166 --> predicate 'award received'

In [133]:
# Find name - award received/nominated for - type of Noble  

In [134]:
queryString = """
SELECT ?person ?pn ?ar ?nameNoble
WHERE { 
 

?person ?p ?noble . 
?noble  wdt:P279 wd:Q7191 .

# this returns the labels
   ?person <http://schema.org/name> ?pn .
    ?p <http://schema.org/name> ?ar .
    ?noble <http://schema.org/name> ?nameNoble .
    
    FILTER(?p = wdt:P166 || ?p = wdt:P1411 )
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q135640'), ('pn', 'Valery Larbaud'), ('ar', 'nominated for'), ('nameNoble', 'Nobel Prize in Literature')]
[('person', 'http://www.wikidata.org/entity/Q335187'), ('pn', 'Edward Grey, 1st Viscount Grey of Fallodon'), ('ar', 'nominated for'), ('nameNoble', 'Nobel Peace Prize')]
[('person', 'http://www.wikidata.org/entity/Q115448'), ('pn', 'Daniel Bovet'), ('ar', 'nominated for'), ('nameNoble', 'Nobel Prize in Physiology or Medicine')]
[('person', 'http://www.wikidata.org/entity/Q16012577'), ('pn', 'Howard Cary'), ('ar', 'nominated for'), ('nameNoble', 'Nobel Prize in Chemistry')]
[('person', 'http://www.wikidata.org/entity/Q2475967'), ('pn', 'Adrien Albert Marie de Mun'), ('ar', 'nominated for'), ('nameNoble', 'Nobel Prize in Literature')]
[('person', 'http://www.wikidata.org/entity/Q216398'), ('pn', 'Alan Sillitoe'), ('ar', 'nominated for'), ('nameNoble', 'Nobel Prize in Literature')]
[('person', 'http://www.wikidata.org/entity/Q164469'

10

In [ ]:
# Get a person IRI random to find its relevant attributes
# Get wd:Q57074 

In [149]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
wd:Q57074 ?p ?o. 


# this returns the labels
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('pname', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('pname', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('pname', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('pname', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pname', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('pname', 'NUKAT ID')]


10

In [ ]:
# GET:
# wdt:P27 --> Country
# wdt:P569 --> date of birthday
# wdt:P21 --> sex or gender
# wdt:P106 --> occupation

In [ ]:
# Find all noble lauterates and nominees with their relevant attributes relevant

In [178]:
queryString = """
SELECT ?person ?pn ?ar ?nameNoble ?countryname ?dateofbirthday ?sexorgender (GROUP_CONCAT(DISTINCT ?occupation ; separator = ", ") AS ?occupations)
WHERE { 
 

?person ?p ?noble .
OPTIONAL { 
   ?person wdt:P27 ?country;
           wdt:P569 ?dateofbirthday ;
           wdt:P21 ?sog;
           wdt:P106 ?oc }
        .
?noble  wdt:P279 wd:Q7191 .

# this returns the labels
   ?person <http://schema.org/name> ?pn .
    ?p <http://schema.org/name> ?ar .
    ?noble <http://schema.org/name> ?nameNoble .
     ?country <http://schema.org/name> ?countryname .
    ?sog <http://schema.org/name> ?sexorgender .
    ?oc <http://schema.org/name> ?occupation .
    FILTER(?p = wdt:P166 || ?p = wdt:P1411 ) 
    
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1385071'), ('pn', 'Robert Serber'), ('ar', 'nominated for'), ('nameNoble', 'Nobel Prize in Physics'), ('countryname', 'United States of America'), ('dateofbirthday', '1909-03-14T00:00:00Z'), ('sexorgender', 'male'), ('occupations', 'nuclear physicist, physicist, university teacher')]
[('person', 'http://www.wikidata.org/entity/Q121372'), ('pn', 'Karl Friedrich Meyer'), ('ar', 'nominated for'), ('nameNoble', 'Nobel Prize in Physiology or Medicine'), ('countryname', 'United States of America'), ('dateofbirthday', '1884-05-19T00:00:00Z'), ('sexorgender', 'male'), ('occupations', 'pathologist, university teacher, veterinarian')]
[('person', 'http://www.wikidata.org/entity/Q109721'), ('pn', 'Dale T. Mortensen'), ('ar', 'award received'), ('nameNoble', 'Prize in Economic Sciences in Memory of Alfred Nobel'), ('countryname', 'United States of America'), ('dateofbirthday', '1939-02-02T00:00:00Z'), ('sexorgender', 'male'), ('occupations', 'ec

10

In [1]:
###

# Workload 2

In [ ]:
# 2. Get the number of Italian, French and Iranian winners and check which nation won more awards in the last twenty years.

In [ ]:
# Find the IRI for France and Iran, starting from Italy IRI wd:Q38

In [152]:
queryString = """
SELECT ?p ?pname
WHERE { 

wd:Q38 wdt:P31 ?p .

# this returns the labels
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q3624078'), ('pname', 'sovereign state')]
[('p', 'http://www.wikidata.org/entity/Q6256'), ('pname', 'country')]


2

In [ ]:
# Get node 'sovereign state' --> wd:Q3624078 to find all the states (its instance of)

In [153]:
queryString = """
SELECT ?s ?state
WHERE { 

?s wdt:P31 wd:Q3624078 .

# this returns the labels
    ?s <http://schema.org/name> ?state .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q756617'), ('state', 'Danish Realm')]
[('s', 'http://www.wikidata.org/entity/Q786'), ('state', 'Dominican Republic')]
[('s', 'http://www.wikidata.org/entity/Q30747910'), ('state', 'Islamic Republic of Afghanistan')]
[('s', 'http://www.wikidata.org/entity/Q1000'), ('state', 'Gabon')]
[('s', 'http://www.wikidata.org/entity/Q1005'), ('state', 'The Gambia')]
[('s', 'http://www.wikidata.org/entity/Q1006'), ('state', 'Guinea')]
[('s', 'http://www.wikidata.org/entity/Q1007'), ('state', 'Guinea-Bissau')]
[('s', 'http://www.wikidata.org/entity/Q1008'), ('state', 'Ivory Coast')]
[('s', 'http://www.wikidata.org/entity/Q1009'), ('state', 'Cameroon')]
[('s', 'http://www.wikidata.org/entity/Q1011'), ('state', 'Cape Verde')]


10

In [ ]:
# Look for France and Iran

In [154]:
queryString = """
SELECT ?s ?state
WHERE { 

?s wdt:P31 wd:Q3624078 .

# this returns the labels
    ?s <http://schema.org/name> ?state .
    
    FILTER (REGEX(?state, "rance") || REGEX(?state, "ran"))
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('state', 'France')]
[('s', 'http://www.wikidata.org/entity/Q794'), ('state', 'Iran')]
[('s', 'http://www.wikidata.org/entity/Q4689103'), ('state', 'Transitional Islamic State of Afghanistan')]


3

In [ ]:
# Get node 'France' --> wd:Q142
# Get node 'Iran' --> wd:Q794

In [ ]:
# To find all the Noble winners we use the predicate found above 'award received' --> wdt:P166
# Try for italian winners

In [229]:
queryString = """
SELECT ?winner ?winnername ?ar ?noblename ?countryname
WHERE { 

    ?winner wdt:P27 wd:Q38 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 


# this returns the labels
    ?winner <http://schema.org/name> ?winnername .
    wdt:P166 <http://schema.org/name> ?ar .
    ?noble <http://schema.org/name> ?noblename .
    wd:Q38 <http://schema.org/name> ?countryname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q115448'), ('winnername', 'Daniel Bovet'), ('ar', 'award received'), ('noblename', 'Nobel Prize in Physiology or Medicine'), ('countryname', 'Italy')]
[('winner', 'http://www.wikidata.org/entity/Q109553'), ('winnername', 'Renato Dulbecco'), ('ar', 'award received'), ('noblename', 'Nobel Prize in Physiology or Medicine'), ('countryname', 'Italy')]
[('winner', 'http://www.wikidata.org/entity/Q186481'), ('winnername', 'Riccardo Giacconi'), ('ar', 'award received'), ('noblename', 'Nobel Prize in Physics'), ('countryname', 'Italy')]
[('winner', 'http://www.wikidata.org/entity/Q173028'), ('winnername', 'Emilio G. Segrè'), ('ar', 'award received'), ('noblename', 'Nobel Prize in Physics'), ('countryname', 'Italy')]
[('winner', 'http://www.wikidata.org/entity/Q234145'), ('winnername', 'Giulio Natta'), ('ar', 'award received'), ('noblename', 'Nobel Prize in Chemistry'), ('countryname', 'Italy')]
[('winner', 'http://www.wikidata.org/entity/Q8753

13

In [ ]:
# Find the numbers of italian, french and iranian winners

In [234]:
queryString = """
SELECT DISTINCT ?italianwinners ?frenchwinners ?iranianwinners
WHERE { 

    
{
    select distinct (COUNT(?winner) AS ?italianwinners) {
    ?winner wdt:P27 wd:Q38 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 
    }
}

{
    select distinct (COUNT(?winner) AS ?frenchwinners) {
    ?winner wdt:P27 wd:Q142 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 
    }
}

{
    select distinct (COUNT(?winner) AS ?iranianwinners) {
    ?winner wdt:P27 wd:Q794 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 
    }
}

# this returns the labels
    
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('italianwinners', '13'), ('frenchwinners', '71'), ('iranianwinners', '1')]


1

In [ ]:
# capire come ricavare date vittoria per stampare numero vincitori degli ultimi 20 anni

In [2]:
#
#

# Workload 3

In [224]:
# 3. Determine how many Literature nobel laureates have the following contries: Italy, Germany, France, Romania, Denmark, 
#    Iran, and China

In [ ]:
# We have to find the IRI for Germany, Romania, Denmark, China
# We have found above the following:
# Italy --> wd:Q38
# France --> wd:Q142
# Iran --> wd:Q794

In [ ]:
# We look now for: Germany, Romania, Denmark, China --> searching on Sovereign state

In [237]:
queryString = """
SELECT ?s ?state
WHERE { 

?s wdt:P31 wd:Q3624078 .

# this returns the labels
    ?s <http://schema.org/name> ?state .
    
    FILTER (REGEX(?state, "erman") || REGEX(?state, "omani") || REGEX(?state, "enmark") || REGEX(?state, "hina"))
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q183'), ('state', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q7318'), ('state', 'Nazi Germany')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('state', 'Romania')]
[('s', 'http://www.wikidata.org/entity/Q35'), ('state', 'Denmark')]
[('s', 'http://www.wikidata.org/entity/Q148'), ('state', "People's Republic of China")]
[('s', 'http://www.wikidata.org/entity/Q107312248'), ('state', 'Republic of China')]


6

In [ ]:
# GET:
# Germany --> wd:Q183
# Romania --> wd:Q218
# Denmark --> wd:Q35
# China --> wd:Q148 

In [ ]:
# Find now the number of Literature Nobel lauraeates have all the countries

In [238]:
queryString = """
SELECT DISTINCT ?italianwinners ?frenchwinners ?iranianwinners ?germanywinners ?romanianwinners ?chinawinners
WHERE { 

    
{
    select distinct (COUNT(?winner) AS ?italianwinners) {
    ?winner wdt:P27 wd:Q38 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 
    }
}

{
    select distinct (COUNT(?winner) AS ?frenchwinners) {
    ?winner wdt:P27 wd:Q142 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 
    }
}

{
    select distinct (COUNT(?winner) AS ?iranianwinners) {
    ?winner wdt:P27 wd:Q794 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 
    }
}

{
    select distinct (COUNT(?winner) AS ?germanywinners) {
    ?winner wdt:P27 wd:Q183 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 
    }
}

{
    select distinct (COUNT(?winner) AS ?romanianwinners) {
    ?winner wdt:P27 wd:Q218 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 
    }
}

{
    select distinct (COUNT(?winner) AS ?denmarkwinners) {
    ?winner wdt:P27 wd:Q35 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 
    }
}

{
    select distinct (COUNT(?winner) AS ?chinawinners) {
    ?winner wdt:P27 wd:Q148 .
    ?winner wdt:P166 ?noble .
    ?noble  wdt:P279 wd:Q7191 . 
    }
}


# this returns the labels
    
} 


LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('italianwinners', '13'), ('frenchwinners', '71'), ('iranianwinners', '1'), ('germanywinners', '74'), ('romanianwinners', '4'), ('chinawinners', '6')]


1

In [7]:
#
#

# Workload 4

In [ ]:
# 4. How many Literature Nobel award winners have a PhD (aka Doctorate of Philosophy)
#    (you may check if they have a doctoral advisor)?

In [8]:
# I look for find PhD IRI to have all doctorate

In [27]:
queryString = """
SELECT DISTINCT ?prof ?profname
WHERE { 
 
 ?d  wdt:P106 ?prof   . 

# this returns the labels
    ?prof <http://schema.org/name> ?profname .

FILTER (REGEX(?profname, "PhD") or REGEX(?profname, "Doctorate of Philosophy"))
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('prof', 'http://www.wikidata.org/entity/Q19603972'), ('profname', 'PhD in Philosophical Sciences')]
[('prof', 'http://www.wikidata.org/entity/Q65640233'), ('profname', 'PhD in philosophy')]


2

In [12]:
# GET:  'PhD in philosophy' --> wd:Q65640233

In [13]:
# Look for all PhD in philosophy

In [26]:
queryString = """
SELECT DISTINCT ?d ?dname
WHERE { 
 
 ?d  wdt:P106 wd:Q65640233  . 

# this returns the labels
    ?d <http://schema.org/name> ?dname .

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('d', 'http://www.wikidata.org/entity/Q89410064'), ('dname', 'Yiota Vassi')]


1

In [ ]:
# ...only one PhD found

In [ ]:
# In this way I can look for all the Phd in philosophy and explore if they win something

In [ ]:
# To find all the Noble winners we use the predicate found above 'award received' --> wdt:P166

In [25]:
queryString = """
SELECT DISTINCT ?winner ?winnername ?ar ?noblename
WHERE { 

    ?winner  wdt:P106 wd:Q65640233 ;
             wdt:P166 ?noble .
    ?noble   wdt:P279 wd:Q7191 . 


# this returns the labels
    ?winner <http://schema.org/name> ?winnername .
    wdt:P166 <http://schema.org/name> ?ar .
    ?noble <http://schema.org/name> ?noblename .
  #  wd:Q37922 <http://schema.org/name> ?noblename .
  
  FILTER(?noble = wd:Q37922)
    
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [ ]:
# Not found anything

In [ ]:
#
#

# Workload 5

In [20]:
# 5. Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? 
#    (if so, who is the author with more books not in the Vatical Library)?

In [ ]:
# I try to find something that won an award 

In [24]:
queryString = """
SELECT DISTINCT ?winner ?winnername ?ar ?noblename
WHERE { 

    ?winner  wdt:P166 ?noble ;
             wdt:P31 ?og .
    ?noble   wdt:P279 wd:Q7191 . 


# this returns the labels
    ?winner <http://schema.org/name> ?winnername .
    wdt:P166 <http://schema.org/name> ?ar .
    ?noble <http://schema.org/name> ?noblename .
  #  wd:Q37922 <http://schema.org/name> ?noblename .
  
  FILTER(?og != wd:Q5)
    
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q42970'), ('winnername', 'Amnesty International'), ('ar', 'award received'), ('noblename', 'Nobel Peace Prize')]
[('winner', 'http://www.wikidata.org/entity/Q5987345'), ('winnername', 'International Committee of the Red Cross'), ('ar', 'award received'), ('noblename', 'Nobel Peace Prize')]
[('winner', 'http://www.wikidata.org/entity/Q1069175'), ('winnername', 'International Federation of Red Cross and Red Crescent Societies'), ('ar', 'award received'), ('noblename', 'Nobel Peace Prize')]
[('winner', 'http://www.wikidata.org/entity/Q1065'), ('winnername', 'United Nations'), ('ar', 'award received'), ('noblename', 'Nobel Peace Prize')]
[('winner', 'http://www.wikidata.org/entity/Q170208'), ('winnername', 'Quakers'), ('ar', 'award received'), ('noblename', 'Nobel Peace Prize')]
[('winner', 'http://www.wikidata.org/entity/Q41984'), ('winnername', 'International Atomic Energy Agency'), ('ar', 'award received'), ('noblename', 'Nobel Peace P

10

In [ ]:
# But I understand that I have to find book.. so I look for instance to find book

In [23]:
queryString = """
SELECT DISTINCT ?og ?ogname 
WHERE { 

    ?winner  wdt:P31 ?og .

# this returns the labels
    ?winner <http://schema.org/name> ?winnername .
    ?og <http://schema.org/name> ?ogname .
  
    
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('og', 'http://www.wikidata.org/entity/Q21814441'), ('ogname', 'ambassador to South Korea')]
[('og', 'http://www.wikidata.org/entity/Q29511881'), ('ogname', 'ambassador of Canada')]
[('og', 'http://www.wikidata.org/entity/Q299931'), ('ogname', 'Chile Open')]
[('og', 'http://www.wikidata.org/entity/Q30057159'), ('ogname', 'ambassador to Israel')]
[('og', 'http://www.wikidata.org/entity/Q40243936'), ('ogname', 'ambassador to the Marshall Islands')]
[('og', 'http://www.wikidata.org/entity/Q104027338'), ('ogname', 'Russian folk game')]
[('og', 'http://www.wikidata.org/entity/Q30057139'), ('ogname', 'ambassador to Uruguay')]
[('og', 'http://www.wikidata.org/entity/Q38152077'), ('ogname', 'ambassador to the Comoros')]
[('og', 'http://www.wikidata.org/entity/Q3342913'), ('ogname', 'Basque family name')]
[('og', 'http://www.wikidata.org/entity/Q4663385'), ('ogname', 'former railway station')]


10

In [ ]:
# Look for 'Book'

In [ ]:
queryString = """
SELECT DISTINCT ?og ?ogname 
WHERE { 

    ?winner  wdt:P31 ?og .
    


# this returns the labels
    ?winner <http://schema.org/name> ?winnername .
    ?og <http://schema.org/name> ?ogname .
  
FILTER regex(?ogname, "\\bbook\\b", "i")  
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results


In [ ]:
# GET: 'book' --> wd:Q571

In [ ]:
# I look only for books that won Noble in Literature

In [6]:
queryString = """
SELECT DISTINCT ?winner ?winnername ?ar ?noblename
WHERE { 

    ?winner  wdt:P166 ?noble ;
             wdt:P279 wd:Q571 .
    ?noble   wdt:P279 wd:Q7191 . 


# this returns the labels
    ?winner <http://schema.org/name> ?winnername .
    wdt:P166 <http://schema.org/name> ?ar .
    ?noble <http://schema.org/name> ?noblename .
  #  wd:Q37922 <http://schema.org/name> ?noblename .
  
 
    
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
Empty


0